In [ ]:
import pandas as pd
import requests
import json
import time
import random
import base64
from IPython.display import display, HTML
from datetime import date



In [ ]:
# Enter the Poshmark Username to be scraped

Poshmark_Username = "chaoscollection"


In [ ]:
def ProPoshScraper(pro):
    print(pro)
    pd.set_option('display.max_columns', 100)

    MIN_SLEEP_TIME = 2
    page = 1

    proUrlBase = "https://poshmark.com/vm-rest/users/"
    proUsername = pro
    proFilter = "/posts/filtered?request=%7B%22filters%22%3A%7B%22"
    proDepartment = "department%22%3A%22All%22%2C%22"
    proInventory = "inventory_status%22%3A%5B%22"
    # all, available, sold_out
    Inventory = "all"
    proURLTail1 = "%22%5D%7D%2C%22query_and_facet_filters%22%3A%7B%22creator_id%22%3A%22"
    proURLTail2 = "%22%7D%2C%22experience%22%3A%22all%22%2C%22"
    proMaxID = "max_id%22%3A"
    MaxIDTail = "%2C%22"
    proCount = "count%22%3A"
    proCount2 = 48
    proURLTail3 = "%7D&summarize=true&app_version=2.55&pm_version=129.0.0"

    start_url = proUrlBase + proUsername + proFilter + proDepartment + proInventory + proURLTail1 + proUsername \
                + proURLTail2 + proCount + str(proCount2) + proURLTail3


    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/"
                             "58.0.3029.110 Safari/537.36 SE 2.X MetaSr 1.0"}

    proxies = {"http": "http://123.207.96.189:80"}

    def get_table_from_url(url):
        response = requests.get(url, proxies=proxies, headers=headers)
        if response.status_code != 200:
            print("Couldn't Connect to Poshmark")
        else:
            text = response.text
            nested_text = json.loads(text)
            # print(nested_text)
            if "statusCode" in nested_text:
                if nested_text["statusCode"] == 404:
                    print("Couldn't find Poshmark Username")
            elif "data" in nested_text:
                df_packet = pd.json_normalize(nested_text['data'])
                # total_items = more_data['total']
                try:
                    if 'more' in nested_text:
                        more_data = nested_text['more']
                        if "next_max_id" in more_data:
                            NewMaxID = str(more_data['next_max_id'])
                            new_url = proUrlBase + proUsername + proFilter + proDepartment + proInventory + proURLTail1 + \
                                      proUsername + proURLTail2 + proMaxID + NewMaxID + MaxIDTail + proCount + \
                                      str(proCount2) + proURLTail3
                        else:
                            new_url = "end"
                    else:
                        new_url = "end"
                except:
                    new_url = "end"

                return df_packet, new_url
    try:
        full_table, next_url = get_table_from_url(start_url)
    except:
        print("Userdata not collected")

    if full_table.size > 0:
        print("Table Size: ",full_table.size)
        while next_url != "end":
            print(str(page) + " " + proUsername + " " + str(full_table.shape[0]) + ' items information collected')
            table_to_add, next_url = get_table_from_url(next_url)
            full_table = pd.concat([full_table, table_to_add])
            sleep_time = MIN_SLEEP_TIME + random.random() * 10
            time.sleep(int(sleep_time))
            page = page + 1

        first_columns = ['id','brand','title','department.display','category','category_v2.display',
                         'category_v2.slug','first_published_at','condition','price','original_price_amount.val',
                         'original_price_amount.currency_code','size_obj.display_with_size_set','size_obj.size_system',
                         'picture_url',
                         'inventory.status','inventory.status_changed_at']
        table_min = full_table[first_columns]

        output_name = 'Poshmark_' + \
                      proUsername + \
                      "_" + \
                      str(table_min.shape[0]) + \
                      " " + \
                      Inventory + \
                      ' items_' + \
                      str(date.today()) + \
                      '.csv'
        return table_min,output_name
    else:
        print("No userdata collected")


## https://www.codegrepper.com/code-examples/python/download+csv+file+through+jupyter+notebook
def create_download_link( df, file):
    title = "Download CSV file"
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=file)
    display(HTML(html))
    return HTML(html)

try:
    results, filename = ProPoshScraper(Poshmark_Username)
    create_download_link(df=results,file=filename)
except:
    print("No data available for csv output")

chaoscollection
Table Size:  2280
